In [1]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
% matplotlib inline
plt.style.use('ggplot')

import seaborn as sns

import cartopy.crs as ccrs
import cartopy.feature as cfeat
import cartopy.io.img_tiles as cimgt

import AISMapPlot as aismap

In [2]:
# Load the data
names = ['TripID', 'MMSI', 'StartLatitude', 'StartLongitude', 'StartTime', 'EndLatitude', 'EndLongitude', 'EndTime',
         'StartPort', 'EndPort', 'ID', 'time', 'shiptype', 'Length', 'Breadth', 'Draught', 'Latitude', 'Longitude',
         'SOG', 'COG', 'TH', 'Destination', 'Name', 'Callsign', 'AisSourcen']
ais = pd.read_csv('rotterdam_hamburg.arff', names = names, skiprows = 27, parse_dates = True,
                  na_values = ['?'], dtype = {'TripID': str, 'MMSI': str, 'shiptype': str})

In [3]:
ais.head()

,TripID,MMSI,StartLatitude,StartLongitude,StartTime,EndLatitude,EndLongitude,EndTime,StartPort,EndPort,...,Draught,Latitude,Longitude,SOG,COG,TH,Destination,Name,Callsign,AisSourcen
0,21109,210481000,51.94,4.05,'2016-1-21 04:32',53.53,9.91,'2016-1-22 06:04',ROTTERDAM,HAMBURG,...,10.64,53.95,7.82,14.7,81.0,80.0,HAMBURG,CMA.CGM.SAMBHAR,5BND2,H7001.DAIS1.81b.71.71a
1,21109,210481000,51.94,4.05,'2016-1-21 04:32',53.53,9.91,'2016-1-22 06:04',ROTTERDAM,HAMBURG,...,10.64,53.95,7.83,14.7,80.0,80.0,HAMBURG,CMA.CGM.SAMBHAR,5BND2,H7001.DAIS1.81b.71.71a
2,21109,210481000,51.94,4.05,'2016-1-21 04:32',53.53,9.91,'2016-1-22 06:04',ROTTERDAM,HAMBURG,...,10.64,53.95,7.83,14.7,81.0,80.0,HAMBURG,CMA.CGM.SAMBHAR,5BND2,H7001.DAIS1.81b.71.71a
3,21109,210481000,51.94,4.05,'2016-1-21 04:32',53.53,9.91,'2016-1-22 06:04',ROTTERDAM,HAMBURG,...,10.64,53.95,7.84,14.7,80.0,80.0,HAMBURG,CMA.CGM.SAMBHAR,5BND2,H7001.DAIS1.81b.71.71a
4,21109,210481000,51.94,4.05,'2016-1-21 04:32',53.53,9.91,'2016-1-22 06:04',ROTTERDAM,HAMBURG,...,10.64,53.95,7.85,14.7,81.0,80.0,HAMBURG,CMA.CGM.SAMBHAR,5BND2,H7001.DAIS1.81b.71.71a


In [4]:
ais_nan = ais

In [5]:
# Convert time columns to correct dtype
ais_nan['StartTime'] = pd.to_datetime(ais_nan['StartTime'], format = '\'%Y-%m-%d %H:%M\'')
ais_nan['EndTime'] = pd.to_datetime(ais_nan['EndTime'], format = '\'%Y-%m-%d %H:%M\'')
ais_nan['time'] = pd.to_datetime(ais_nan['time'], format = '\'%Y-%m-%d %H:%M\'')

In [6]:
# Convert all headings that are 511 (>= 360) to NaN
ais_nan.loc[ais_nan['TH'] >= 360, 'TH'] = np.nan

In [7]:
# Convert courses >= 360 to NaN
ais_nan.loc[ais_nan['COG'] >= 360, 'COG'] = np.nan

In [8]:
# Set invalid shiptypes to NaN
# Invalid shiptypes existing in the data set are '0' and '159'
ais_nan.loc[(ais_nan['shiptype'] == '0') | (ais_nan['shiptype'] == '159'), 'shiptype'] = np.nan

In [9]:
# Set invalid lengths (0 or > 400) to NaN
ais_nan.loc[(ais_nan['Length'] <= 0) | (ais_nan['Length'] > 400), 'Length'] = np.nan

In [10]:
# Set invalid breadths (0 or > 59) to NaN
ais_nan.loc[(ais_nan['Breadth'] <= 0) | (ais_nan['Breadth'] > 59), 'Breadth'] = np.nan

In [11]:
# Set speeds that are unrealisticly high to NaN
ais_nan.loc[ais_nan['SOG'] > 25.6, 'SOG'] = np.nan

In [12]:
ais_dropped = ais_nan.drop(['AisSourcen', 'ID', 'StartPort', 'EndPort', 'Name', 'MMSI', 'shiptype'], axis = 1)

In [ ]:
# Plot all positions in the data before any filters are applied
#aismap.plot_to_map(size = (13, 13), longitude = ais_dropped['Longitude'], latitude = ais_dropped['Latitude'])
#plt.title("All Positions Before Filtering")

In [ ]:
# Plot start positions before any filtering to map
#aismap.plot_to_map(size = (8, 8), longitude = ais_dropped['StartLongitude'], latitude = ais_dropped['StartLatitude'])
#plt.title("Start Positions Before Filtering")

In [ ]:
# Plot end positions before any filtering to map
#aismap.plot_to_map(size = (8, 8), longitude = ais_dropped['EndLongitude'], latitude = ais_dropped['EndLatitude'])
#plt.title("End Positions Before Filtering")

In [13]:
# Get rid of trips that leave our zone (2.45 - 10.66 / 51.49 - 55.06)

outside_long_mask = (ais_dropped['Longitude'] < 2.45) | (ais_dropped['Longitude'] > 10.66)
outside_lat_mask = (ais_dropped['Latitude'] < 51.49) | (ais_dropped['Latitude'] > 55.06)

outside_rec_mask = outside_long_mask | outside_lat_mask

outside_trip_ids = ais_dropped['TripID'][outside_rec_mask].unique()
outside_trip_mask = ~ais_dropped['TripID'].isin(outside_trip_ids)
ais_zoned = ais_dropped[outside_trip_mask]

In [ ]:
#aismap.plot_to_map(size = (13, 13), longitude = ais_zoned['Longitude'], latitude = ais_zoned['Latitude'])
#plt.title("All Positions After Filtering to Our Zone")

In [ ]:
#minlat = ais_zoned['EndLatitude'].min() - 0.2
#maxlat = ais_zoned['EndLatitude'].max() + 0.2
#minlong = ais_zoned['EndLongitude'].min() - 0.2
#maxlong = ais_zoned['EndLongitude'].max() + 0.2

#aismap.plot_google_map((minlong, maxlong, minlat, maxlat))
#plt.scatter(x = ais_zoned['EndLongitude'], y = ais_zoned['EndLatitude'] , transform = ccrs.PlateCarree(), s = 1)

In [14]:
# Get times finish line is crossed, by trips that do cross it
finish_line_ts = ais_zoned[ais_zoned['Longitude'] > 9.81].groupby('TripID')['time'].min()

# Only keep trips that do cross the finish line
ais_ham_zoned = ais_zoned[ais_zoned['TripID'].isin(finish_line_ts.index)]

#Set EndTime to the time the ship first appears behind the finish line
for tripid in finish_line_ts.index :
    ais_ham_zoned.loc[ais_ham_zoned['TripID'] == tripid, 'EndTime'] = finish_line_ts.loc[tripid]

# Throw out all records after the finish line was crossed
ais_ham_zoned = ais_ham_zoned[ais_ham_zoned['time'] <= ais_ham_zoned['EndTime']]

/Users/jankaiser/anaconda2/lib/python2.7/site-packages/pandas/core/indexing.py:537: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


In [ ]:
#aismap.plot_to_map(size = (13, 13), longitude = ais_ham_zoned['Longitude'], latitude = ais_ham_zoned['Latitude'])

In [15]:
ais_tts = ais_ham_zoned
ais_tts.is_copy = False   # Hack to silence SettingWithCopyWarning

# Add time for the entire trip to the data
ais_tts['TravelTime'] = ais_tts['EndTime'] - ais_tts['StartTime']
ais_tts['TravelTimeMins'] = ais_tts['TravelTime'].transform(lambda x : x.total_seconds() / 60)

# Add time remaining until arrival at destination
ais_tts['remainingTT'] = ais_tts['EndTime'] - ais_tts['time']
ais_tts['remainingMins'] = ais_tts['remainingTT'].transform(lambda x : x.total_seconds() / 60)

In [ ]:
#travel_times = ais_tts.groupby('TripID')['TravelTime'].max()
#travel_times = travel_times.transform(lambda x : x.total_seconds() // 3600)
#travel_times.value_counts().sort_index().plot.bar(figsize = (25, 10))

In [ ]:
#travel_times[travel_times < travel_times.quantile(.75)].value_counts().sort_index().plot.bar(figsize = (25, 10))

In [16]:
# Only keep majority of shorter trips
ais_shortmaj = ais_tts[ais_tts['TravelTime'] < ais_tts['TravelTime'].quantile(.75)]

In [ ]:
#aismap.col_plot_to_map(size = (13, 13), longitude = ais_shortmaj['Longitude'], latitude = ais_shortmaj['Latitude'], col= ais_shortmaj['remainingMins'])

In [ ]:
# Get and plot the longest trip in the data set
#longest_trip = ais_shortmaj[ais_shortmaj['TravelTime'] == ais_shortmaj['TravelTime'].max()]
#longest_trip.head()

In [ ]:
#aismap.col_plot_to_map(size = (13, 13), longitude = longest_trip['Longitude'], latitude = longest_trip['Latitude'], col = longest_trip['remainingMins'])
#plt.title("Longest Trip in the Data Set")

In [17]:
ais_tfeats = ais_shortmaj
ais_tfeats.is_copy = False

ais_tfeats['hour_of_day'] = ais_tfeats['time'].dt.hour
ais_tfeats['weekday'] = ais_tfeats['time'].dt.dayofweek
ais_tfeats['day_of_month'] = ais_tfeats['time'].dt.day
ais_tfeats['month'] = ais_tfeats['time'].dt.month

In [ ]:
ais_interpol = ais_tfeats.copy()
ais_interpol.index = ais_interpol['time']
ais_interpol = ais_interpol.groupby('TripID').apply(lambda x : x.resample('T').mean().interpolate(method='linear')).reset_index()

In [ ]:
#ais_tfeats.groupby('weekday')['TravelTimeMins'].mean().plot(figsize = (15, 5))

In [76]:
# Assign sector IDs according to Longitude ranges (5 sectors for ROT-HAM route)
def assign_sectors(ais_before):
    ais_after = ais_before.copy()
    
    ais_after.loc[(ais_after['Longitude'] <= 4.8) & (ais_after['Latitude'] <= 52.8), 'sector'] = '0'
    ais_after.loc[(ais_after['Longitude'] <= 4.8) & (ais_after['Latitude'] > 52.8), 'sector'] = '1'
    ais_after.loc[(ais_after['Longitude'] > 4.8) & (ais_after['Longitude'] <= 6.0), 'sector'] = '2'
    ais_after.loc[(ais_after['Longitude'] > 6.0) & (ais_after['Longitude'] <= 7.2), 'sector'] = '3'
    ais_after.loc[(ais_after['Longitude'] > 7.2) & (ais_after['Longitude'] <= 8.6), 'sector'] = '4'
    ais_after.loc[(ais_after['Longitude'] > 8.6) & (ais_after['Longitude'] <= 9.81), 'sector'] = '5'
    ais_after.loc[ais_after['Longitude'] > 9.81, 'sector'] = np.nan
    
    return ais_after

ais_sectors = assign_sectors(ais_tfeats)

In [ ]:
#aismap.col_plot_to_map(size = (13, 13), longitude = ais_sectors['Longitude'], latitude = ais_sectors['Latitude'], col = ais_sectors['sector_id'])

In [79]:
ais_secleav = ais_sectors.copy()
ais_secleav = ais_secleav.sort_values('time')

ais_secleav['next_lat'] = ais_secleav.groupby('TripID')['Latitude'].shift(-1)
ais_secleav['next_lon'] = ais_secleav.groupby('TripID')['Longitude'].shift(-1)
ais_secleav['next_time'] = ais_secleav.groupby('TripID')['time'].shift(-1)
ais_secleav['next_sector_id'] = ais_secleav.groupby('TripID')['sector'].shift(-1)
ais_secleav = ais_secleav.sort_values(['TripID', 'time'])
ais_secleav['next_trip_id'] = ais_secleav['TripID'].shift(-1)

lasts = ais_secleav[ais_secleav['sector'] != ais_secleav['next_sector_id']]

lasts.loc[lasts['sector'] == '0', 'sector_leave_lat'] = 52.8
lasts.loc[lasts['sector'] == '1', 'sector_leave_lon'] = 4.8
lasts.loc[lasts['sector'] == '2', 'sector_leave_lon'] = 6.0
lasts.loc[lasts['sector'] == '3', 'sector_leave_lon'] = 7.2
lasts.loc[lasts['sector'] == '4', 'sector_leave_lon'] = 8.6
lasts.loc[lasts['sector'] == '5', 'sector_leave_lon'] = 9.81

lasts.loc[lasts['sector'] == '0', 'sector_leave_lon'] = lasts.loc[lasts['sector'] == '0', 'Longitude'] + ((lasts.loc[lasts['sector'] == '0', 'sector_leave_lat'] - lasts.loc[lasts['sector'] == '0', 'Latitude']) / (lasts.loc[lasts['sector'] == '0', 'next_lat'] - lasts.loc[lasts['sector'] == '0', 'Latitude'])) * (lasts.loc[lasts['sector'] == '0', 'next_lat'] - lasts.loc[lasts['sector'] == '0', 'Latitude'])
lasts.loc[lasts['sector'] == '1', 'sector_leave_lat'] = lasts.loc[lasts['sector'] == '1', 'Latitude'] + ((lasts.loc[lasts['sector'] == '1', 'sector_leave_lon'] - lasts.loc[lasts['sector'] == '1', 'Longitude']) / (lasts.loc[lasts['sector'] == '1', 'next_lon'] - lasts.loc[lasts['sector'] == '1', 'Longitude'])) * (lasts.loc[lasts['sector'] == '1', 'next_lon'] - lasts.loc[lasts['sector'] == '1', 'Longitude'])
lasts.loc[lasts['sector'] == '2', 'sector_leave_lat'] = lasts.loc[lasts['sector'] == '2', 'Latitude'] + ((lasts.loc[lasts['sector'] == '2', 'sector_leave_lon'] - lasts.loc[lasts['sector'] == '2', 'Longitude']) / (lasts.loc[lasts['sector'] == '2', 'next_lon'] - lasts.loc[lasts['sector'] == '2', 'Longitude'])) * (lasts.loc[lasts['sector'] == '2', 'next_lon'] - lasts.loc[lasts['sector'] == '2', 'Longitude'])
lasts.loc[lasts['sector'] == '3', 'sector_leave_lat'] = lasts.loc[lasts['sector'] == '3', 'Latitude'] + ((lasts.loc[lasts['sector'] == '3', 'sector_leave_lon'] - lasts.loc[lasts['sector'] == '3', 'Longitude']) / (lasts.loc[lasts['sector'] == '3', 'next_lon'] - lasts.loc[lasts['sector'] == '3', 'Longitude'])) * (lasts.loc[lasts['sector'] == '3', 'next_lon'] - lasts.loc[lasts['sector'] == '3', 'Longitude'])
lasts.loc[lasts['sector'] == '4', 'sector_leave_lat'] = lasts.loc[lasts['sector'] == '4', 'Latitude'] + ((lasts.loc[lasts['sector'] == '4', 'sector_leave_lon'] - lasts.loc[lasts['sector'] == '4', 'Longitude']) / (lasts.loc[lasts['sector'] == '4', 'next_lon'] - lasts.loc[lasts['sector'] == '4', 'Longitude'])) * (lasts.loc[lasts['sector'] == '4', 'next_lon'] - lasts.loc[lasts['sector'] == '4', 'Longitude'])
lasts.loc[lasts['sector'] == '5', 'sector_leave_lat'] = lasts.loc[lasts['sector'] == '5', 'Latitude'] + ((lasts.loc[lasts['sector'] == '5', 'sector_leave_lon'] - lasts.loc[lasts['sector'] == '5', 'Longitude']) / (lasts.loc[lasts['sector'] == '5', 'next_lon'] - lasts.loc[lasts['sector'] == '5', 'Longitude'])) * (lasts.loc[lasts['sector'] == '5', 'next_lon'] - lasts.loc[lasts['sector'] == '5', 'Longitude'])

lasts.loc[lasts['sector'] == '0', 'sector_leave_time'] = lasts.loc[lasts['sector'] == '0', 'time'] + ((lasts.loc[lasts['sector'] == '0', 'sector_leave_lat'] - lasts.loc[lasts['sector'] == '0', 'Latitude']) / (lasts.loc[lasts['sector'] == '0', 'next_lat'] - lasts.loc[lasts['sector'] == '0', 'Latitude'])) * (lasts.loc[lasts['sector'] == '0', 'next_time'] - lasts.loc[lasts['sector'] == '0', 'time'])
lasts.loc[lasts['sector'] == '1', 'sector_leave_time'] = lasts.loc[lasts['sector'] == '1', 'time'] + ((lasts.loc[lasts['sector'] == '1', 'sector_leave_lon'] - lasts.loc[lasts['sector'] == '1', 'Longitude']) / (lasts.loc[lasts['sector'] == '1', 'next_lon'] - lasts.loc[lasts['sector'] == '1', 'Longitude'])) * (lasts.loc[lasts['sector'] == '1', 'next_time'] - lasts.loc[lasts['sector'] == '1', 'time'])
lasts.loc[lasts['sector'] == '2', 'sector_leave_time'] = lasts.loc[lasts['sector'] == '2', 'time'] + ((lasts.loc[lasts['sector'] == '2', 'sector_leave_lon'] - lasts.loc[lasts['sector'] == '2', 'Longitude']) / (lasts.loc[lasts['sector'] == '2', 'next_lon'] - lasts.loc[lasts['sector'] == '2', 'Longitude'])) * (lasts.loc[lasts['sector'] == '2', 'next_time'] - lasts.loc[lasts['sector'] == '2', 'time'])
lasts.loc[lasts['sector'] == '3', 'sector_leave_time'] = lasts.loc[lasts['sector'] == '3', 'time'] + ((lasts.loc[lasts['sector'] == '3', 'sector_leave_lon'] - lasts.loc[lasts['sector'] == '3', 'Longitude']) / (lasts.loc[lasts['sector'] == '3', 'next_lon'] - lasts.loc[lasts['sector'] == '3', 'Longitude'])) * (lasts.loc[lasts['sector'] == '3', 'next_time'] - lasts.loc[lasts['sector'] == '3', 'time'])
lasts.loc[lasts['sector'] == '4', 'sector_leave_time'] = lasts.loc[lasts['sector'] == '4', 'time'] + ((lasts.loc[lasts['sector'] == '4', 'sector_leave_lon'] - lasts.loc[lasts['sector'] == '4', 'Longitude']) / (lasts.loc[lasts['sector'] == '4', 'next_lon'] - lasts.loc[lasts['sector'] == '4', 'Longitude'])) * (lasts.loc[lasts['sector'] == '4', 'next_time'] - lasts.loc[lasts['sector'] == '4', 'time'])
lasts.loc[lasts['sector'] == '5', 'sector_leave_time'] = lasts.loc[lasts['sector'] == '5', 'time'] + ((lasts.loc[lasts['sector'] == '5', 'sector_leave_lon'] - lasts.loc[lasts['sector'] == '5', 'Longitude']) / (lasts.loc[lasts['sector'] == '5', 'next_lon'] - lasts.loc[lasts['sector'] == '5', 'Longitude'])) * (lasts.loc[lasts['sector'] == '5', 'next_time'] - lasts.loc[lasts['sector'] == '5', 'time'])

ais_secleav['sector_leave_lat'] = lasts['sector_leave_lat']
ais_secleav['sector_leave_lon'] = lasts['sector_leave_lon']
ais_secleav['sector_leave_time'] = lasts['sector_leave_time']

ais_secleav['sector_leave_lat'] = ais_secleav.groupby(['TripID', 'sector'])['sector_leave_lat'].transform('last')
ais_secleav['sector_leave_lon'] = ais_secleav.groupby(['TripID', 'sector'])['sector_leave_lon'].transform('last')
ais_secleav['sector_leave_time'] = ais_secleav.groupby(['TripID', 'sector'])['sector_leave_time'].transform('last')

ais_secleav = ais_secleav.drop(['next_lat', 'next_lon', 'next_time', 'next_sector_id', 'next_trip_id'], axis = 1)

In [80]:
ais_secleav_ts = ais_secleav.copy()
ais_secleav_ts['time_to_sector_leave'] = ais_secleav_ts['sector_leave_time'] - ais_secleav_ts['time']
ais_secleav_ts['mins_to_sector_leave'] = ais_secleav_ts['time_to_sector_leave'].transform(lambda x: x.total_seconds() / 60)

In [81]:
ais_secleav_ts.columns

Index([u'TripID', u'StartLatitude', u'StartLongitude', u'StartTime',
       u'EndLatitude', u'EndLongitude', u'EndTime', u'time', u'Length',
       u'Breadth', u'Draught', u'Latitude', u'Longitude', u'SOG', u'COG',
       u'TH', u'Destination', u'Callsign', u'TravelTime', u'TravelTimeMins',
       u'remainingTT', u'remainingMins', u'hour_of_day', u'weekday',
       u'day_of_month', u'month', u'sector', u'sector_leave_lat',
       u'sector_leave_lon', u'sector_leave_time', u'time_to_sector_leave',
       u'mins_to_sector_leave'],
      dtype='object')

In [84]:
learners = ['TripID', 'time', 'Length', 'Breadth', 'Draught', 'Latitude', 'Longitude',
            'SOG', 'COG', 'TH', 'TravelTimeMins', 'remainingMins', 'hour_of_day',
            'weekday', 'day_of_month', 'month', 'sector', 'mins_to_sector_leave', 'sector_leave_lat',
            'sector_leave_lon']

In [85]:
ais_learners = ais_secleav_ts[learners]
ais_learners = ais_learners.dropna()
ais_learners.head()

,TripID,time,Length,Breadth,Draught,Latitude,Longitude,SOG,COG,TH,TravelTimeMins,remainingMins,hour_of_day,weekday,day_of_month,month,sector,mins_to_sector_leave,sector_leave_lat,sector_leave_lon
834543,1000226,2016-06-15 14:57:00,369.0,51.0,13.8,51.97,4.03,5.4,112.3,112.0,1494.0,1494.0,14,2,15,6,0,301.0,52.8,4.2
834550,1000226,2016-06-15 14:58:00,369.0,51.0,13.8,51.97,4.03,5.2,112.4,112.0,1494.0,1493.0,14,2,15,6,0,300.0,52.8,4.2
834566,1000226,2016-06-15 14:59:00,369.0,51.0,13.8,51.97,4.04,5.0,113.2,112.0,1494.0,1492.0,14,2,15,6,0,299.0,52.8,4.2
834343,1000226,2016-06-15 15:00:00,369.0,51.0,13.8,51.96,4.04,4.8,112.8,112.0,1494.0,1491.0,15,2,15,6,0,298.0,52.8,4.2
834348,1000226,2016-06-15 15:01:00,369.0,51.0,13.8,51.96,4.04,4.8,111.2,111.0,1494.0,1490.0,15,2,15,6,0,297.0,52.8,4.2


In [86]:
# Export without downsampling
ais_learners = ais_secleav_ts[learners]
ais_learners = ais_learners.dropna()
ais_learners.to_pickle('ROT-HAM_learners_01min.pkl')

In [90]:
# Export downsampled to 5 minutes
ais_downsampled = ais_secleav_ts.copy()
ais_downsampled.index = ais_downsampled['time']
ais_downsampled = ais_downsampled.groupby('TripID').apply(lambda x : x.resample('5T').mean()).reset_index()
ais_downsampled = assign_sectors(ais_downsampled)

ais_learners = ais_downsampled[learners]
ais_learners = ais_learners.dropna()
ais_learners.to_pickle('ROT-HAM_learners_05min.pkl')

In [91]:
# Export downsampled to 10 minutes
ais_downsampled = ais_secleav_ts.copy()
ais_downsampled.index = ais_downsampled['time']
ais_downsampled = ais_downsampled.groupby('TripID').apply(lambda x : x.resample('10T').mean()).reset_index()
ais_downsampled = assign_sectors(ais_downsampled)

ais_learners = ais_downsampled[learners]
ais_learners = ais_learners.dropna()
ais_learners.to_pickle('ROT-HAM_learners_10min.pkl')

In [93]:
# Export downsampled to 15 minutes
ais_downsampled = ais_secleav_ts.copy()
ais_downsampled.index = ais_downsampled['time']
ais_downsampled = ais_downsampled.groupby('TripID').apply(lambda x : x.resample('15T').mean()).reset_index()
ais_downsampled = assign_sectors(ais_downsampled)

ais_learners = ais_downsampled[learners]
ais_learners = ais_learners.dropna()
ais_learners.to_pickle('ROT-HAM_learners_15min.pkl')

In [95]:
# Export downsampled to 30 minutes
ais_downsampled = ais_secleav_ts.copy()
ais_downsampled.index = ais_downsampled['time']
ais_downsampled = ais_downsampled.groupby('TripID').apply(lambda x : x.resample('30T').mean()).reset_index()
ais_downsampled = assign_sectors(ais_downsampled)

ais_learners = ais_downsampled[learners]
ais_learners = ais_learners.dropna()
ais_learners.to_pickle('ROT-HAM_learners_30min.pkl')

In [97]:
# Export downsampled to 60 minutes
ais_downsampled = ais_secleav_ts.copy()
ais_downsampled.index = ais_downsampled['time']
ais_downsampled = ais_downsampled.groupby('TripID').apply(lambda x : x.resample('60T').mean()).reset_index()
ais_downsampled = assign_sectors(ais_downsampled)

ais_learners = ais_downsampled[learners]
ais_learners = ais_learners.dropna()
ais_learners.to_pickle('ROT-HAM_learners_60min.pkl')